## CNN & Preprocessing for MNIST

Использовать сверточную нейронную сеть и препроцессинг для распознавания цифр в наборе данных MNIST.

Получить точность распознавания больше 99%.

In [1]:
from keras.datasets import mnist

from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras import models
from keras import layers
import keras

Using TensorFlow backend.


#### Набор данных

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print('Train:', x_train.shape, y_train.shape)
print('Test: ', x_test.shape, y_test.shape)

Train: (60000, 28, 28) (60000,)
Test:  (10000, 28, 28) (10000,)


Нормализуем значения пикселей изображений в градациях серого до диапазона [0, 1]. Для этого преобразуем тип данных из целых чисел без знака в числа с плавающей запятой, а затем поделим значений пикселей на максимальное значение.

Для выходных данных преобразуем целое число в двоичный вектор из 10 элементов. 

In [3]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

x_train = x_train.astype('float32') 
x_test = x_test.astype('float32')

x_train /= 255.0
x_test /= 255.0

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

#### Предобработка данных

Создадим экзепляр класса ImageDataGenerator для автоматического расширения данных при обучении модели.

Используем zoom_range для создания случайного масштабирования изображения в диапазоне [0.9, 1.1]. Используем rotation_range для случайного поворота изображения.

In [7]:
datagen = ImageDataGenerator(
    zoom_range=(0.9, 1.1),
    rotation_range=20,
    validation_split = 0.2
)

iterator = datagen.flow(x_train, y_train)

#### Convolutional Neural Network

In [9]:
model = models.Sequential()

model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(10, activation='softmax'))

model.compile(
    optimizer='adam', 
    metrics=['accuracy'], 
    loss='categorical_crossentropy'
)

model.fit_generator(iterator, epochs=8)

score = model.evaluate(x_test, y_test, verbose=0)

print('Loss=', score[0]) 
print('Accuracy=', score[1]) 

Epoch 1/8
1875/1875 [==============================] - 20s 11ms/step - loss: 0.1827 - accuracy: 0.9422
Epoch 2/8
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0684 - accuracy: 0.9788
Epoch 3/8
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0530 - accuracy: 0.9842
Epoch 4/8
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0443 - accuracy: 0.9865
Epoch 5/8
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0390 - accuracy: 0.9879
Epoch 6/8
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0351 - accuracy: 0.9894
Epoch 7/8
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0307 - accuracy: 0.9904
Epoch 8/8
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0276 - accuracy: 0.9917
Loss= 0.02164245426790585
Accuracy= 0.9939000010490417
